# Human Resources Analytics Dataset Analysis

Module: CS985: Machine Learning For Data Analytics

Student: Rokas Labeikis

Student ID: 201349799

## Objective and problem


## Introduction to the dataset


## Obtaining the dataset

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import tensorflow as tf
import tensorflow_hub as hub
warnings.resetwarnings()
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import glob
import os
import cv2
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import platform

/home/rokas/anaconda2/lib/python2.7/site-packages/sklearn/utils/__init__.py:10: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .murmurhash import murmurhash3_32
/home/rokas/anaconda2/lib/python2.7/site-packages/scipy/stats/_continuous_distns.py:17: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _stats
/home/rokas/anaconda2/lib/python2.7/site-packages/sklearn/utils/extmath.py:24: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._logistic_sigmoid import _log_logistic_sigmoid
/home/rokas/anaconda2/lib/python2.7/site-packages/sklearn/utils/extmath.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .sparsefuncs_fast import csr_row_norms


In [3]:
print('TensorFlow version: ', tf.__version__)
print('Python version: ', platform.python_version())

('TensorFlow version: ', '1.7.0')
('Python version: ', '2.7.13')


In [37]:
train_dir = Path('input/training/')
test_dir = Path('input/validation/')
test_file = './input/test-paths-labels.csv'
train_file = './input/train-paths-labels.csv'

In [3]:
#label info
info = pd.read_csv("input/monkey_labels.txt", names=['Label','Latin Name', 'Common Name','Train Images', 'Validation Images'], skiprows=1)
info

,Label,Latin Name,Common Name,Train Images,Validation Images
0,n0,alouatta_palliata\t,mantled_howler,131,26
1,n1,erythrocebus_patas\t,patas_monkey,139,28
2,n2,cacajao_calvus\t,bald_uakari,137,27
3,n3,macaca_fuscata\t,japanese_macaque,152,30
4,n4,cebuella_pygmea\t,pygmy_marmoset,131,26
5,n5,cebus_capucinus\t,white_headed_capuchin,141,28
6,n6,mico_argentatus\t,silvery_marmoset,132,26
7,n7,saimiri_sciureus\t,common_squirrel_monkey,142,28
8,n8,aotus_nigriceps\t,black_headed_night_monkey,133,27
9,n9,trachypithecus_johnii,nilgiri_langur,132,26


In [148]:
def read_data(filename):
    images = []
    labels = []
    i = 0
    for training_dir_path in glob.glob(filename + "/*"):
        label = info["Common Name"][i].rstrip()
        i=i+1
        for image_path in glob.glob(os.path.join(training_dir_path, "*.jpg")):
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)
            image = cv2.resize(image, (1, 1))
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
            labels.append(label)
            images.append(image)
            paths.append(image_path)
            
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(labels)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False)
    i_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    
    d = {'images':images,'labels':labels, 'int':integer_encoded, 'hot': onehot_encoded.tolist()}

    df = pd.DataFrame(d)
    return df

train = read_data("input/training")
test = read_data("input/validation")
train.to_csv('./input/train.csv', index=False) 
test.to_csv('./input/test.csv',  index=False)



In [ ]:
# We know images are 150 by 150 
img_size = 150

# Images are stored in one-dimensional arrays of this length
img_size_flat = img_size * img_size

# Tuple with height and width of images used to reshape arrays
img_shape = (img_size, img_size)

# Number of classes
num_classes = 10

def plot_images(images, cls_true, cls_pred=None):
    assert len(images) == len(cls_true) == 9
    
    # Create figure with 3x3 sub-plots.
    fig, axes = plt.subplots(3, 3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        # Plot image.
        ax.imshow(images[i])

        # Show true and predicted classes.
        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true[i])
        else:
            xlabel = "True: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])

        ax.set_xlabel(xlabel)
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
        
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()
    


# Get the first images from the test-set.
images = train["images"][0:9]

# Get the true classes for those images.
cls_true = train["labels"][0:9]

# Plot the images and labels using our helper-function above.
plot_images(images=images, cls_true=cls_true)


In [6]:
print(train.columns)
print(train.head())
print(train.shape)
print(test.shape)


# Sample image
plt.imshow(train["images"][200])
plt.show()
print(train["labels"][200])

NameError: name 'train' is not defined

## Preparing the dataset

## Exploring the dataset

## Creating a model

### Transfer learning

In [9]:
# python label_image.py --graph=retrained_model/output_graph.pb --labels=retrained_model/output_labels.txt --input_layer=Placeholder --output_layer=final_result --image=input/validation/n9/n901.jpg

def load_labels(label_file):
    label = []
    proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
    for l in proto_as_ascii_lines:
        label.append(l.rstrip())
    return label

def read_tensor_from_image_file(file_name,
                                input_height=299,
                                input_width=299,
                                input_mean=0,
                                input_std=255):
    input_name = "file_reader"
    output_name = "normalized"
    file_reader = tf.read_file(file_name, input_name)
   
    image_reader = tf.image.decode_jpeg(file_reader, channels=3, name="jpeg_reader")
    
    float_caster = tf.cast(image_reader, tf.float32)
    dims_expander = tf.expand_dims(float_caster, 0)
    resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
    normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
    sess = tf.Session()
    result = sess.run(normalized)
    
    return result

model_file = 'retrained_model/output_graph.pb'
label_file = 'retrained_model/output_labels.txt'
input_layer = 'Placeholder'
output_layer = 'final_result'

#load graph
graph = tf.Graph()
graph_def = tf.GraphDef()
with open(model_file, "rb") as f:
    graph_def.ParseFromString(f.read())
with graph.as_default():
    tf.import_graph_def(graph_def)

t = read_tensor_from_image_file('input/validation/n2/n201.jpg')

input_name = "import/" + input_layer
output_name = "import/" + output_layer
input_operation = graph.get_operation_by_name(input_name)
output_operation = graph.get_operation_by_name(output_name)

with tf.Session(graph=graph) as sess:
    results = sess.run(output_operation.outputs[0], {
        input_operation.outputs[0]: t
    })
results = np.squeeze(results)

top_k = results.argsort()[-5:][::-1]
labels = load_labels(label_file)
for i in top_k:
    print(labels[i], results[i])

('bald_uakari', 0.95616525)
('japanese_macaque', 0.02068729)
('nilgiri_langur', 0.007473636)
('common_squirrel_monkey', 0.0067563313)
('black_headed_night_monkey', 0.0023083456)


## Linear model

In [30]:
def generate_labelled_input_fn(input_file, batch_size):
    def input_fn():
        def read_labeled_image_list(image_list_file):
            df = pd.read_csv(image_list_file)
            return df["path"].tolist(), df["int"].tolist()

        image_list, label_list = read_labeled_image_list(input_file)

        def read_images_from_disk(input_queue):
            label = input_queue[1]
            file_contents = tf.read_file(input_queue[0])
            example = tf.image.decode_jpeg(file_contents, channels=3)
            example=tf.image.resize_images(example, [150, 150])
            example=tf.reshape(example, [150, 150, 3]) 
            return example, label

        images = tf.convert_to_tensor(image_list, dtype=tf.string)
        labels = tf.convert_to_tensor(label_list, dtype=tf.int32)

        # Makes an input queue
        input_queue = tf.train.slice_input_producer([images, labels], shuffle=True)

        image, label = read_images_from_disk(input_queue)
        image_batch, label_batch = tf.train.batch([image, label], batch_size=batch_size)
        return {'images':  image_batch}, label_batch
    return input_fn

In [31]:
# image_arrays = np.asarray(train['images'], dtype=np.float32).reshape((1, 1))
feature_columns = [tf.feature_column.numeric_column('images', shape=[150,150,3])]

classifier = tf.estimator.LinearClassifier(
    feature_columns=feature_columns,
    n_classes=10,
    model_dir='./linear_model'
)

BATCH_SIZE = 40
TRAIN_STEPS = 2000

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f221178b4d0>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': './linear_model', '_save_summary_steps': 100}


In [32]:
classifier.train(
    input_fn=generate_labelled_input_fn('./input/train-paths-labels.csv', BATCH_SIZE),
    steps=TRAIN_STEPS
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./linear_model/model.ckpt.
INFO:tensorflow:loss = 92.10341, step = 1
INFO:tensorflow:global_step/sec: 1.50712
INFO:tensorflow:loss = 2125745.5, step = 101 (66.353 sec)
INFO:tensorflow:global_step/sec: 1.4876
INFO:tensorflow:loss = 1353128.0, step = 201 (67.224 sec)
INFO:tensorflow:global_step/sec: 1.27107
INFO:tensorflow:loss = 590059.5, step = 301 (78.673 sec)
INFO:tensorflow:global_step/sec: 1.33276
INFO:tensorflow:loss = 272023.0, step = 401 (75.031 sec)
INFO:tensorflow:global_step/sec: 1.19312
INFO:tensorflow:loss = 216931.0, step = 501 (83.815 sec)
INFO:tensorflow:global_step/sec: 1.22351
INFO:tensorflow:loss = 277782.25, step = 601 (81.733 sec)
INFO:tensorflow:global_step/sec: 1.21044
INFO:tensorflow:l

In [36]:
classifier.evaluate(
    input_fn=generate_labelled_input_fn('./input/test-paths-labels.csv', BATCH_SIZE),
    steps=100
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-11-16:56:17
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./linear_model/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-04-11-16:57:28
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.54725, average_loss = 14453.339, global_step = 2000, loss = 578133.56


{'accuracy': 0.54725,
 'average_loss': 14453.339,
 'global_step': 2000,
 'loss': 578133.56}

### Convolutional Neural Networks

## Validation

### Confusion matrix

## Evaluation

## Conclusion 